In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
driver = webdriver.Chrome()
url = "https://www.transfermarkt.com/premier-league/tabelle/wettbewerb/GB1/saison_id/2023"
driver.get(url)

In [3]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
# Wait for the page to load
time.sleep(5)  # Adjust as needed for your internet speed

In [5]:
table = soup.find('table', {'class':'items'})
rows = table.find_all('tr')

In [6]:
position = []
teams = []
played_games = []
wins = []
draws = []
losses = []
goals = []
goal_diff = []
points = []
team_ids = []

In [7]:
for row in rows[1:]:  # Skip the header row
    cols = row.find_all('td')
    if len(cols) > 0:
        position.append(cols[0].text.strip())  # Position
        teams.append(cols[2].text.strip())  # Team name (adjusted index)
        played_games.append(42)  # Games played (constant value)
        wins.append(cols[4].text.strip())  # Wins
        draws.append(cols[5].text.strip())  # Draws
        losses.append(cols[6].text.strip())  # Losses
        goals.append(cols[7].text.strip())  # Goals (For:Against)
        goal_diff.append(cols[8].text.strip())  # Goal difference (adjusted index)
        points.append(cols[9].text.strip())  # Points (adjusted index)

In [8]:
df = pd.DataFrame({
    'Position': position,
    'Team': teams,
    'Played': played_games,
    'Wins': wins,
    'Draws': draws,
    'Losses': losses,
    'Goals': goals,
    'Goal Difference': goal_diff,
    'Points': points
})

In [9]:
df.head(23)

,Position,Team,Played,Wins,Draws,Losses,Goals,Goal Difference,Points
0,1,Man City,42,28,7,3,96:34,62,91
1,2,Arsenal,42,28,5,5,91:29,62,89
2,3,Liverpool,42,24,10,4,86:41,45,82
3,4,Aston Villa,42,20,8,10,76:61,15,68
4,5,Tottenham,42,20,6,12,74:61,13,66
5,6,Chelsea,42,18,9,11,77:63,14,63
6,7,Newcastle,42,18,6,14,85:62,23,60
7,8,Man Utd,42,18,6,14,57:58,-1,60
8,9,West Ham,42,14,10,14,60:74,-14,52
9,10,Crystal Palace,42,13,10,15,57:58,-1,49


In [10]:
df.replace({'Man City': 'Manchester City', 'Man Utd': 'Manchester United', 'Sheff Utd': 'Sheffield United', 'Sheff Wed': 'Sheffield Wednesday', 'Nottm Forest': 'Nottingham Forest', 'QPR': 'Queens Park Rangers', 'Wolves': 'Wolverhampton Wanderers', 'Leeds': 'Leeds United', 'Ipswich': 'Ipswich Town', 'Coventry': 'Coventry City', 'Wimbledon FC': 'Wimbledon', 'Tottenham': 'Tottenham Hotspur', 'Norwich': 'Norwich City', 'Blackburn': 'Blackburn Rovers', 'Newcastle': 'Newcastle United', 'West Ham': 'West Ham United', 'Leicester': 'Leicester City', 'Bolton': 'Bolton Wanderers', 'Derby': 'Derby County', 'Barnsley FC': 'Barnsley', 'Charlton': 'Charlton Athletic', 'Bradford': 'Bradford City', 'West Brom': 'West Bromwich Albion', 'Birmingham': 'Birmingham City', 'Wolves': 'Wolverhampton Wanderers', 'Wigan': 'Wigan Athletic', 'Swansea': 'Swansea City', 'Cardiff': 'Cardiff City', 'Huddersfield': 'Huddersfield Town', 'Brighton': 'Brighton & Hove Albion', 'Luton': 'Luton Town'}, inplace=True)

In [11]:
# Close the driver
driver.quit()

In [12]:
df.head()

,Position,Team,Played,Wins,Draws,Losses,Goals,Goal Difference,Points
0,1,Manchester City,42,28,7,3,96:34,62,91
1,2,Arsenal,42,28,5,5,91:29,62,89
2,3,Liverpool,42,24,10,4,86:41,45,82
3,4,Aston Villa,42,20,8,10,76:61,15,68
4,5,Tottenham Hotspur,42,20,6,12,74:61,13,66


In [13]:
df2 = pd.read_csv("EPL Matches 1992_1993_2023_2024.csv", index_col=0)

In [14]:
def add_managers_to_league_table(matches_df, league_table_df, season):
    """
    Add a column to the league table dataframe with managers who managed each team during a given season.
    
    Args:
        matches_df (pd.DataFrame): The dataframe containing match data.
        league_table_df (pd.DataFrame): The league table dataframe.
        season (str): The season to look for managers (e.g., '1992/1993').
    
    Returns:
        pd.DataFrame: The updated league table dataframe with a new 'Manager' column.
    """
    # Filter matches for the provided season
    season_matches = matches_df[matches_df['Season'] == season]

    # Create an empty dictionary to hold team names and their corresponding managers
    team_managers = {}

    # Loop through each match to get managers for the home and away teams
    for _, match in season_matches.iterrows():
        # For Home Team
        home_team = match['Home_Team']
        home_manager = match['Manager_Home']
        
        if home_team not in team_managers:
            team_managers[home_team] = set()  # Use a set to avoid duplicates
        team_managers[home_team].add(home_manager)

        # For Away Team
        away_team = match['Away_Team']
        away_manager = match['Manager_Away']
        
        if away_team not in team_managers:
            team_managers[away_team] = set()  # Use a set to avoid duplicates
        team_managers[away_team].add(away_manager)
    
    # Now, for each team in the league table, find their managers and add to the new column
    managers_list = []
    
    for _, row in league_table_df.iterrows():
        team_name = row['Team']
        
        if team_name in team_managers:
            # Convert the set of managers to a sorted list, then to a string
            managers = ', '.join(sorted(team_managers[team_name]))
        else:
            managers = 'Unknown'
        
        managers_list.append(managers)

    # Add the new column to the league table dataframe
    league_table_df['Manager'] = managers_list

    return league_table_df

In [15]:
updated_league_table = add_managers_to_league_table(df2, df, '2023/2024')
updated_league_table.head(23)

,Position,Team,Played,Wins,Draws,Losses,Goals,Goal Difference,Points,Manager
0,1,Manchester City,42,28,7,3,96:34,62,91,Pep Guardiola
1,2,Arsenal,42,28,5,5,91:29,62,89,Mikel Arteta
2,3,Liverpool,42,24,10,4,86:41,45,82,Jurgen Klopp
3,4,Aston Villa,42,20,8,10,76:61,15,68,Unai Emery
4,5,Tottenham Hotspur,42,20,6,12,74:61,13,66,Ange Postecoglou
5,6,Chelsea,42,18,9,11,77:63,14,63,Mauricio Pochettino
6,7,Newcastle United,42,18,6,14,85:62,23,60,Eddie Howe
7,8,Manchester United,42,18,6,14,57:58,-1,60,Erik ten Hag
8,9,West Ham United,42,14,10,14,60:74,-14,52,David Moyes
9,10,Crystal Palace,42,13,10,15,57:58,-1,49,"Oliver Glasner, Ray Lewington, Roy Hodgson"


In [16]:
df_manager = pd.read_csv("Manager History.csv")
team_id_mapping = dict(zip(df_manager['Team_Name'], df_manager['Team_ID']))
updated_league_table['Team_ID'] = updated_league_table['Team'].map(team_id_mapping)
updated_league_table.head(23)

,Position,Team,Played,Wins,Draws,Losses,Goals,Goal Difference,Points,Manager,Team_ID
0,1,Manchester City,42,28,7,3,96:34,62,91,Pep Guardiola,MCI
1,2,Arsenal,42,28,5,5,91:29,62,89,Mikel Arteta,ARS
2,3,Liverpool,42,24,10,4,86:41,45,82,Jurgen Klopp,LIV
3,4,Aston Villa,42,20,8,10,76:61,15,68,Unai Emery,AVI
4,5,Tottenham Hotspur,42,20,6,12,74:61,13,66,Ange Postecoglou,THO
5,6,Chelsea,42,18,9,11,77:63,14,63,Mauricio Pochettino,CHE
6,7,Newcastle United,42,18,6,14,85:62,23,60,Eddie Howe,NUN
7,8,Manchester United,42,18,6,14,57:58,-1,60,Erik ten Hag,MUN
8,9,West Ham United,42,14,10,14,60:74,-14,52,David Moyes,WHA
9,10,Crystal Palace,42,13,10,15,57:58,-1,49,"Oliver Glasner, Ray Lewington, Roy Hodgson",CPA


In [17]:
updated_league_table['Season'] = '2023/2024'
updated_league_table = updated_league_table[['Season', 'Position', 'Team_ID', 'Team', 'Played', 'Wins', 'Draws', 'Losses', 'Goals', 'Goal Difference', 'Points', 'Manager']]
updated_league_table.head(23)

,Season,Position,Team_ID,Team,Played,Wins,Draws,Losses,Goals,Goal Difference,Points,Manager
0,2023/2024,1,MCI,Manchester City,42,28,7,3,96:34,62,91,Pep Guardiola
1,2023/2024,2,ARS,Arsenal,42,28,5,5,91:29,62,89,Mikel Arteta
2,2023/2024,3,LIV,Liverpool,42,24,10,4,86:41,45,82,Jurgen Klopp
3,2023/2024,4,AVI,Aston Villa,42,20,8,10,76:61,15,68,Unai Emery
4,2023/2024,5,THO,Tottenham Hotspur,42,20,6,12,74:61,13,66,Ange Postecoglou
5,2023/2024,6,CHE,Chelsea,42,18,9,11,77:63,14,63,Mauricio Pochettino
6,2023/2024,7,NUN,Newcastle United,42,18,6,14,85:62,23,60,Eddie Howe
7,2023/2024,8,MUN,Manchester United,42,18,6,14,57:58,-1,60,Erik ten Hag
8,2023/2024,9,WHA,West Ham United,42,14,10,14,60:74,-14,52,David Moyes
9,2023/2024,10,CPA,Crystal Palace,42,13,10,15,57:58,-1,49,"Oliver Glasner, Ray Lewington, Roy Hodgson"


In [18]:
updated_league_table.to_csv('2023_2024 League Table.csv')